In [1]:
import tensorflow as tf

import numpy as np
import os
import time
from google.colab import drive

In [2]:
# Get the data from Google Drive
drive.mount('/content/drive')

path_to_file = "/content/drive/My Drive/train.txt"

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Length of text: 111540 characters
3 unique characters


In [3]:
# Vectorize the text
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  'P' :   0,
  'R' :   1,
  'S' :   2,
  ...
}


In [4]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'RRRRRPPPPPSSS' ---- characters mapped to int ---- > [1 1 1 1 1 0 0 0 0 0 2 2 2]


In [5]:
# The maximum length sentence you want for a single input in characters
seq_length = 5
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

R
R
R
R
R


In [6]:
# The batch method lets us easily convert these individual characters to sequences of the desired size.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'RRRRRP'
'PPPPSS'
'SSSRRR'
'RRPPPP'
'PSSSSS'


In [7]:
# For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Print the first example input and target values
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'RRRRR'
Target data: 'RRRRP'


In [8]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 ('R')
  expected output: 1 ('R')
Step    1
  input: 1 ('R')
  expected output: 1 ('R')
Step    2
  input: 1 ('R')
  expected output: 1 ('R')
Step    3
  input: 1 ('R')
  expected output: 1 ('R')
Step    4
  input: 1 ('R')
  expected output: 0 ('P')


In [9]:
# Create training batches
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 5), (64, 5)), types: (tf.int64, tf.int64)>

In [10]:
# Build The Model
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024 * 4

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True, # batch_input_shape=(batch_size, timesteps, data_dim)
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [11]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 5, 3) # (batch_size, sequence_length, vocab_size)


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           768       
_________________________________________________________________
lstm (LSTM)                  (64, None, 4096)          71319552  
_________________________________________________________________
dense (Dense)                (64, None, 3)             12291     
Total params: 71,332,611
Trainable params: 71,332,611
Non-trainable params: 0
_________________________________________________________________


In [13]:
# To get actual predictions from the model you need to sample from the output distribution, to get actual character indices
# This distribution is defined by the logits over the character vocabulary

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

print("Input: \n", repr("".join([idx2char[i] for i in input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'PSSSS'

Next Char Predictions: 
 'RPPSR'


In [14]:
# Train the model
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 5, 3)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       1.098037


In [15]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [16]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/My Drive/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [17]:
# Execute the training
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
290/290 [==============================] - 21s 66ms/step - loss: 0.4616 - accuracy: 0.8028
Epoch 2/10
290/290 [==============================] - 20s 67ms/step - loss: 0.1667 - accuracy: 0.9128
Epoch 3/10
290/290 [==============================] - 20s 68ms/step - loss: 0.1169 - accuracy: 0.9200
Epoch 4/10
290/290 [==============================] - 20s 69ms/step - loss: 0.1154 - accuracy: 0.9202
Epoch 5/10
290/290 [==============================] - 21s 70ms/step - loss: 0.1140 - accuracy: 0.9197
Epoch 6/10
290/290 [==============================] - 21s 71ms/step - loss: 0.1142 - accuracy: 0.9190
Epoch 7/10
290/290 [==============================] - 21s 71ms/step - loss: 0.1147 - accuracy: 0.9194
Epoch 8/10
290/290 [==============================] - 21s 71ms/step - loss: 0.1128 - accuracy: 0.9207
Epoch 9/10
290/290 [==============================] - 21s 71ms/step - loss: 0.1285 - accuracy: 0.9161
Epoch 10/10
290/290 [==============================] - 21s 72ms/step - loss: 0.142

In [18]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            768       
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 4096)           71319552  
_________________________________________________________________
dense_1 (Dense)              (1, None, 3)              12291     
Total params: 71,332,611
Trainable params: 71,332,611
Non-trainable params: 0
_________________________________________________________________


In [19]:
def load_model(checkpoint_dir, vocab_size, embedding_dim, rnn_units):

  model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

  model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

  model.build(tf.TensorShape([1, None]))

  model.summary()

  return model

In [20]:
def generate_text(model, start_string, test_text, help = False, num_generate = 100):
    # Evaluation step (generating text using the learned model)

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Count number of matches
    count = 0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        next_char = idx2char[predicted_id]

        # If network predicted correctly
        if next_char == test_text[i]:
          count += 1

        # If with help, set next char the real one from test text
        if help:
          next_char = test_text[i]

        text_generated.append(next_char)

    return count

In [21]:
import tensorflow as tf
import numpy as np
import os

# Load model
model = load_model(checkpoint_dir, vocab_size, embedding_dim, rnn_units)

# Testing
path_to_file_test = "/content/drive/My Drive/test.txt"

# Read, then decode for py2 compat.
test_text = open(path_to_file_test, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(test_text)))

def test_model(model, test_text, help = False):
  print("Helping network to predict :", help)
  print(generate_text(model, start_string = str(test_text[0:seq_length]), test_text = test_text[seq_length:], help = help, num_generate = len(test_text) - seq_length))

test_model(model, test_text, True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            768       
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 4096)           71319552  
_________________________________________________________________
dense_2 (Dense)              (1, None, 3)              12291     
Total params: 71,332,611
Trainable params: 71,332,611
Non-trainable params: 0
_________________________________________________________________
Length of text: 1110 characters
Helping network to predict : True
403


In [22]:
# Results

# Trained on file generated with step 5 (RRRRRPPPPPSSSSS), accuracy 0.9 (with seq_length = 5)

# Testing on the file where only R appears -> 403/1000
# Testing on file generated with step 5 (RRRRRPPPPPSSSSS) as data for training -> 853/1000

# Conclusion
# If there IS a pattern in training set, with a good seq_length parameter (need to find it experimentally) its possible to learn pattern of player
# Having pattern of player after predicting her/his move we can "show" the opposite figure and win
